In [1]:
import pandas as pd
import altair as alt
import copy
import numpy as np

## Read data

In [2]:
import os
import sys
sys.path.insert(0, '.')

from analyze import *

In [24]:

#dat = pd.read_csv("../examples/exp_results.csv")
dat = pd.read_csv("../tmp_quantile_exp_results.csv")

#dat = alldat[alldat.workload == 'Uniform']
#dat = pd.read_csv("../experiments/exp_results.csv")
print(dat.shape)
#dat

(114000, 14)


In [38]:
dat

,Unnamed: 0,error,answer,truth,query_idx,qid,query,query_parameters,workload,data_seed,sketch_name,sketch_size,sketch_bytes,sketch_seed
0,0,-0.011697,0.038368,0.050065,9999,0,quantile,0.05,Uniform,0,KLL,283,2264,1257003945918453947
1,1,-0.004971,0.096473,0.101444,9999,1,quantile,0.10,Uniform,0,KLL,283,2264,14323118372569728935
2,2,-0.012441,0.134809,0.147250,9999,2,quantile,0.15,Uniform,0,KLL,283,2264,5209562887680129177
3,3,-0.004950,0.189323,0.194273,9999,3,quantile,0.20,Uniform,0,KLL,283,2264,11537272335458328846
4,4,-0.010049,0.230048,0.240096,9999,4,quantile,0.25,Uniform,0,KLL,283,2264,364489192590407506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,22795,-0.500000,75000.500000,75001.000000,99999,185,quantile,0.75,StreamMaker,0,Tdigest,1416,22656,4671844188427764433
113996,22796,-0.500000,80000.500000,80001.000000,99999,186,quantile,0.80,StreamMaker,0,Tdigest,1416,22656,8178879947897898658
113997,22797,-0.500000,85000.500000,85001.000000,99999,187,quantile,0.85,StreamMaker,0,Tdigest,1416,22656,15011068117511139093
113998,22798,-0.500000,90000.500000,90001.000000,99999,188,quantile,0.90,StreamMaker,0,Tdigest,1416,22656,13329689011024994648


In [25]:
dat[['qid', 'workload']].astype(str)

,qid,workload
0,0,Uniform
1,1,Uniform
2,2,Uniform
3,3,Uniform
4,4,Uniform
...,...,...
113995,185,StreamMaker
113996,186,StreamMaker
113997,187,StreamMaker
113998,188,StreamMaker


## Make plots

In [28]:

(chart, _) = plotErrorVsSize(dat[(dat.query_idx > 99000) & (dat['query'] == 'quantile')], x='sketch_size', unit=['qid','workload'])
chart.facet(column='workload').resolve_scale(
    y='independent'
)
#.facet(column='sketch_name').resolve_scale(
#   y='independent'
#)


(855, 21)


alt.FacetChart(...)

In [29]:
(chart, _) = plotErrorVsSize(dat[(dat.query_parameters > 0.90)], x='sketch_size', unit=['qid','workload'])
chart.facet(column='workload').resolve_scale(
    y='independent'
)


(1500, 21)


alt.FacetChart(...)

0          0.60
1          0.75
2          0.35
3          0.15
4          0.05
           ... 
5104725    0.40
5104726    0.50
5104727    0.35
5104728    0.70
5104729    0.60
Name: query_parameters, Length: 5104730, dtype: float64

In [37]:
(chart, aggdat) = plotExperimentResults(dat[(dat['query'] == 'quantile')], x='query_parameters', unit=None, 
                                        column='sketch_name',row='workload', normalize_by='sketch_size', 
                                        normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300,continuousHeight=300).resolve_scale(
    y='independent'
)

alt.FacetChart(...)

In [36]:
(chart, aggdat) = plotExperimentResults(dat[dat['query'] == 'cdf'], x='query_parameters', unit=None, 
                                        column='sketch_name',normalize_by='sketch_size', 
                                        normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300,continuousHeight=300)

alt.FacetChart(...)

In [343]:
(chart, raw_aggdat) = plotBias(dat[dat['query'] == 'quantile'], x='query_parameters', column='sketch_name', unit=None,
                               normalize_by='sketch_size', normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300)


alt.FacetChart(...)

In [289]:
(chart, raw_aggdat) = plotBias(dat[dat['query'] == 'cdf'], x='query_parameters', column='sketch_name', unit=None)
#                               normalize_by='sketch_size', normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300)


alt.FacetChart(...)

In [279]:
(chart, raw_aggdat) = plotBias(dat[dat['query'] == 'quantile'], x='query_idx', column='sketch_name', row='query_parameters', 
                               normalize_by='sketch_size', normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300,continuousHeight=100)


alt.FacetChart(...)

In [255]:
(chart, aggdat) = plotExperimentResults(dat, x='query_idx', column='sketch_name', row='query', 
                                        normalize_by='sketch_size', normalize_by_sqrt=False)
chart.configure_view(continuousWidth=300,continuousHeight=100)

alt.FacetChart(...)

In [339]:
from tdigest import TDigest
from sketches.Sketches import KLLSketch
from numpy.random import random

digest = TDigest(delta=1/10)
s = KLLSketch(size=100)
x = random(10000)
x.sort()
for y in x:
    digest.update(y)
    s.add(y)
print(len(digest.centroids_to_list()), s.getSize())
print(digest.percentile(75), s.query("q", 0.75), x[int(0.75*len(x))])
print(digest.percentile(75.01), s.query("q", 0.7501), x[int(0.7501*len(x))])
print(digest.percentile(75.25), s.query("q", 0.7525), x[int(0.7525*len(x))])
print(digest.percentile(75.5), s.query("q", 0.755), x[int(0.755*len(x))])
print(digest.percentile(75.75), s.query("q", 0.757), x[int(0.7575*len(x))])



206 283
0.7469496205372751 0.7482060194015503 0.7470206101007019
0.7470462610985489 0.7482060194015503 0.7470311395717634
0.7493656345691191 0.7518516182899475 0.7486770922894196
0.7517816486009633 0.7518516182899475 0.7520725370247607
0.7541976626328075 0.7544211745262146 0.7541223416964576


In [334]:
pd.DataFrame(zip(*[(x['m'], x['c']) for x in digest.centroids_to_list()]))
pd.DataFrame((x['m'], x['c']) for x in digest.centroids_to_list())

,0,1
0,1.610343e-04,1.0
1,2.792710e-04,1.0
2,2.815956e-04,1.0
3,5.180549e-04,1.0
4,5.281410e-04,1.0
...,...,...
147,9.999875e-01,1.0
148,1.000000e+06,1.0
149,1.000000e+06,1.0
150,1.000000e+06,1.0


In [337]:
1e6+random(1)

array([1000000.97251855])

In [341]:
for i in range(100):
    digest.update(6*(i**1.5))
digest.centroids_to_list()

[{'m': 0.0, 'c': 2.0},
 {'m': 8.995460964245083e-05, 'c': 1.0},
 {'m': 0.00016742136831093823, 'c': 1.0},
 {'m': 0.0002603745304764482, 'c': 1.0},
 {'m': 0.0003457016256558365, 'c': 1.0},
 {'m': 0.000536813520719881, 'c': 1.0},
 {'m': 0.0006599143576533351, 'c': 1.0},
 {'m': 0.0007519322580852861, 'c': 2.0},
 {'m': 0.0009206696236137568, 'c': 2.0},
 {'m': 0.0009953998249312423, 'c': 2.0},
 {'m': 0.0011625478173844517, 'c': 2.0},
 {'m': 0.0015325742206007131, 'c': 4.0},
 {'m': 0.0017710802325052511, 'c': 5.0},
 {'m': 0.0021716336439111896, 'c': 3.0},
 {'m': 0.002591142421944835, 'c': 7.0},
 {'m': 0.0033530492126616312, 'c': 7.0},
 {'m': 0.00436707040233058, 'c': 16.0},
 {'m': 0.005632118555010534, 'c': 10.0},
 {'m': 0.007504239867735115, 'c': 24.0},
 {'m': 0.010500848700772888, 'c': 32.0},
 {'m': 0.014734950582549589, 'c': 33.0},
 {'m': 0.01779419973822537, 'c': 31.0},
 {'m': 0.022687335664030462, 'c': 48.0},
 {'m': 0.031175260866194533, 'c': 91.0},
 {'m': 0.038608672267552775, 'c': 66.